In [101]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime

In [102]:
# Hàm lưu trữ log
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now().strftime(timestamp_format)
    log_entry = f"{now}: {message}\n"
    with open(r"C:\Users\H\Downloads\FINAL_PROJECT\code_log.txt", "a") as f:
        f.write(log_entry)
    print(log_entry, end = "")

In [103]:
log_progress("LET GO")

2025-Feb-24-16:10:29: LET GO


In [100]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
table_col = ['Name', 'MC_USD_Billion', 'MC_GBP_Billion', 'MC_EUR_Billion', 'MC_INR_Billion']

In [121]:
#Hàm extract dữ liệu
def extract(url):
    html_text = requests.get(url).text
    html_soup = BeautifulSoup(html_text,'html.parser')
    df = pd.DataFrame(columns = table_col)
    tbody = html_soup.find_all('tbody')
    rows = tbody[1].find_all('tr')
    for row in rows:
        cell = row.find_all('td')
        if len(cell) >= 3:
            df.loc[len(df)] = {
                table_col[0]: cell[1].get_text(strip=True),
                table_col[1]: cell[2].get_text(strip=True),  
                table_col[2]: None,
                table_col[3]: None,
                table_col[4]: None
            }
    log_progress('Extract data')
    return df

In [122]:
df = extract(url)

# Kiểm tra dữ liệu
print(df.head())

2025-Feb-24-16:17:44: Extract data
                                              Name MC_USD_Billion  \
0  Industrial and Commercial Bank of China Limited       5,742.86   
1                          China Construction Bank       5,016.81   
2                       Agricultural Bank of China       4,919.03   
3                                    Bank of China       4,192.12   
4                                   JPMorgan Chase       3,868.24   

  MC_GBP_Billion MC_EUR_Billion MC_INR_Billion  
0           None           None           None  
1           None           None           None  
2           None           None           None  
3           None           None           None  
4           None           None           None  


In [110]:
exchange_rate = pd.read_csv(r'C:\Users\H\Downloads\FINAL_PROJECT\exchange_rate.csv')
exchange_dict = exchange_rate.set_index('Currency').to_dict()['Rate']

In [111]:
print(exchange_dict)

{'EUR': 0.93, 'GBP': 0.8, 'INR': 82.95}


In [112]:
# Hàm biến đổi
df[df.columns[1:]] = df[df.columns[1:]].apply(pd.to_numeric)
def transform(df):
    df['MC_GBP_Billion'] = [np.round(x*exchange_dict['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*exchange_dict['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*exchange_dict['INR'],2) for x in df['MC_USD_Billion']]
    log_progress('Transform data')
    return df

In [113]:
df = transform(df)
df.head(10)

2025-Feb-24-16:12:26: Transform data


,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,Bank of America,231.52,185.22,215.31,19204.58
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,HDFC Bank,157.91,126.33,146.86,13098.63
5,Wells Fargo,155.87,124.70,144.96,12929.42
6,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
7,Morgan Stanley,140.83,112.66,130.97,11681.85
8,China Construction Bank,139.82,111.86,130.03,11598.07
9,Bank of China,136.81,109.45,127.23,11348.39


In [114]:
#Hàm chuyển dữ liệu vào file csv
def load_to_csv(df,csv_path):
    df.to_csv(csv_path)
    log_progress('Load to CSV')

In [115]:
load_to_csv(df,r'C:\Users\H\Downloads\FINAL_PROJECT\top_10_banks.csv')

2025-Feb-24-16:12:31: Load to CSV


In [116]:
#Hàm load vào db sqlite
table_name = 'Largest_banks'
db_name = 'Banks.db'
conn = sqlite3.connect(db_name)
def load_to_db(df, conn, table_name):
    df.to_sql(table_name, conn, if_exists = 'replace', index = False)
    log_progress('Load to DB')

In [117]:
load_to_db(df,conn,table_name)

2025-Feb-24-16:12:34: Load to DB


In [123]:
# Hàm chạy các câu truy vấn
def run_query(query_statement,conn):
    #print(query_statement)
    query_output = pd.read_sql(query_statement,conn)
    print(query_output)

In [125]:
run_query(f"SELECT * FROM Largest_banks",conn)

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

In [126]:
run_query(f"SELECT AVG(MC_GBP_Billion) FROM Largest_banks",conn)

   AVG(MC_GBP_Billion)
0              151.987


In [127]:
run_query(f"SELECT Name FROM Largest_banks LIMIT 5",conn)

                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank
